In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/data-sets/Masnavi.csv')

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.utils import np_utils
import numpy as np
import pandas as pd

In [ ]:
f = open("/content/drive/MyDrive/data-sets/Masnavi1.txt", "r")
data = f.read()

In [ ]:
tokenizer = Tokenizer()
def dataset_preparation(data):
    corpus = data.lower().split("\n")    
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = np_utils.to_categorical(label, num_classes=total_words)

    return predictors, label, max_sequence_len, total_words

In [ ]:
"""
Sentence: "they are learning data science"
PREDICTORS             | LABEL
they                   | are
they are               | learning
they are learning      | data
they are learning data | science
"""

'\nSentence: "they are learning data science"\nPREDICTORS             | LABEL\nthey                   | are\nthey are               | learning\nthey are learning      | data\nthey are learning data | science\n'

In [ ]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(150))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(predictors, label, epochs=100, verbose=1)

    return model

In [ ]:
X, Y, max_len, total_words = dataset_preparation(data)
model = create_model(X, Y, max_len, total_words)

Epoch 1/100
1553/1553 [==============================] - 13s 8ms/step - loss: 7.4515
Epoch 2/100
1553/1553 [==============================] - 12s 8ms/step - loss: 7.1319
Epoch 3/100
1553/1553 [==============================] - 11s 7ms/step - loss: 7.0150
Epoch 4/100
1553/1553 [==============================] - 12s 7ms/step - loss: 6.8564
Epoch 5/100
1553/1553 [==============================] - 11s 7ms/step - loss: 6.6832
Epoch 6/100
1553/1553 [==============================] - 12s 8ms/step - loss: 6.5018
Epoch 7/100
1553/1553 [==============================] - 11s 7ms/step - loss: 6.2967
Epoch 8/100
1553/1553 [==============================] - 12s 7ms/step - loss: 6.0699
Epoch 9/100
1553/1553 [==============================] - 11s 7ms/step - loss: 5.8342
Epoch 10/100
1553/1553 [==============================] - 11s 7ms/step - loss: 5.5899
Epoch 11/100
1553/1553 [==============================] - 12s 7ms/step - loss: 5.3467
Epoch 12/100
1553/1553 [==============================] - 11s 7

In [ ]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predict_x = model.predict(token_list, verbose=0)
        predicted = np.argmax(predict_x,axis=1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
# !mkdir -p saved_model
# model.save('masnavi_language_model.h5') 

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 62, 10)            92910     
                                                                 
 lstm_1 (LSTM)               (None, 150)               96600     
                                                                 
 dropout_1 (Dropout)         (None, 150)               0         
                                                                 
 dense_1 (Dense)             (None, 9291)              1402941   
                                                                 
Total params: 1,592,451
Trainable params: 1,592,451
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
seed_text = "گر بريزى بحر"
text = generate_text(seed_text, 10, 50, model)
print(text)

گر بريزى بحر را در كوزه‏اى چند گنجد قسمت يك روزه‏اى‏ يك خلق
